In [1]:
import $ivy.`org.apache.spark::spark-sql:2.4.5` 
import $ivy.`sh.almond::almond-spark:0.9.1`

import $ivy.$                                   

import $ivy.$                              

In [2]:
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)

import org.apache.log4j.{Level, Logger}


In [3]:
import org.apache.spark.sql._

val spark = {
  NotebookSparkSession.builder()
    .master("local[*]")
    .getOrCreate()
}


Loading spark-stubs
Getting spark JARs
Creating SparkSession


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties


Spark UI

import org.apache.spark.sql._


spark: SparkSession = org.apache.spark.sql.SparkSession@7962059c

3 ways to create Spark RDDs
- Parallelized Collections
- External Datasets
- Existig RDDs


RDDs are inmutable


In [4]:
def sc = spark.sparkContext

defined function sc

In [ ]:
// Create an RDD from files

val rdd1 = sc.textFile("data/sample.txt")
rdd1.count()


In [ ]:
// Create and RDD through Parallelized Collection

val rdd = sc.parallelize(1 to 1000, 100)

In [ ]:
// Create an RDD from existing RDDs
val newRDD = rdd.map(data => (data *2))

In [ ]:
// Number of items
newRDD.count()

In [ ]:
// Filter operation - Create a new RDD
val filteredrdd = rdd1.filter(line=>line.contains("11"))
filteredrdd.collect()

In [ ]:
rdd1.count()

In [ ]:
// Read the first item
rdd1.first()

In [ ]:
// REad the first 3 items
rdd1.take(3)

In [ ]:
// Count the number of partitions
rdd1.partitions.length

In [ ]:
// Cache the files
rdd1.cache()

In [ ]:
rdd1.collect()

// Word Count

In [ ]:
val song=sc.textFile("data/eternalflame.txt")

In [ ]:
val wc = song.flatMap(line => line.split(" ")).map(word => (word,1)).reduceByKey(_+_)

In [ ]:
wc.take(5)

In [ ]:
song.collect()

Spark context: 
It allows your Spark Application to access Spark Cluster with the help of Resource Manager.

The resource manager can be one of these three- Spark Standalone, YARN, Apache Mesos.

Only one SparkContext may be active per JVM. You must stop the active it before creating a new one as below:
stop()

A stage is nothing but a step in a physical execution plan

Executors in Spark are worker nodes

RDD -> Resilient Distributed Dataset 

Spark exposes RDD through language integrated API. In integrated API each data set is represented as an object and transformation is involved using the method of these objects.

Lazy Evaluation: Spark computes transformations when an action requires a result for the driver program.



Spark RDD Operations:


 1.  Transformations :  are functions that take an RDD as the input and produce one or many RDDs as the output (Lazy)  e.g. Map(), filter(), reduceByKey() 
   
    -- Narrow transformations : Is the result of map, filter and such that the data is from a single partition only.
    Eg. Map, FlatMAp, MapPartition, Filter, Sample, Union
    
    -- Wide Transformation: Is the resulto of grouppByKey() and reduceByKey() like functions. This could come from multiple partitions.
    
    Eg. Intersect, Distinct, ReduceByKey, GroupByKey, Join, Cartesian, Repartition, Coalesce
    
    
 2. Actions: Returns final results of RDD computations.  An Action is one of the ways to send result from executors to the driver. First(), take(), reduce(), collect(), the count() is some of the Actions in spark.
 
 
 

Unlike Dataframe and datasets, RDDs don’t infer the schema of the ingested data and requires the user to specify it.



In [ ]:
// RDDs on cache
sc.getPersistentRDDs

## RDDs creation
#### 1. Parallelized collection

In [ ]:


val data=spark.sparkContext.parallelize(Seq(("maths",52),("english",75),("science",82), ("computer",65),("maths",85)))
val sorted = data.sortByKey()
sorted.foreach(println)

In [ ]:
data.collect

In [ ]:
// To set the number of partitions (3)

val rdd1 = spark.sparkContext.parallelize(Array("jan","feb","mar","april","may","jun"),3)
// coalesce reduce the number of partitions
val result = rdd1.coalesce(2)
result.foreach(println)

#### 2. External Dataset

In [ ]:
import org.apache.spark.sql.SparkSession
val dataRDD = spark.read.csv("data/sample.csv").rdd
// Here .rdd method is used to convert Dataset<Row> to RDD<Row>.
dataRDD.collect()

In [ ]:
val dataJSONRDD = spark.read.json("data/sample.json").rdd
dataJSONRDD.collect()

In [ ]:
val dataTEXTRDD = spark.read.textFile("data/sample.txt").rdd
dataTEXTRDD.first()

#### 3. Creating RDD from existing RDD

Transformation mutates one RDD into another RDD. Some of the operations applied on RDD are: filter, count, distinct, Map, FlatMap etc.

In [ ]:
val words=spark.sparkContext.parallelize(Seq("the", "quick", "brown", "fox", "jumps", "over", "the", "lazy", "dog"))
val wordPair = words.map(w => (w.charAt(0), w))
wordPair.foreach(println)

## Spark Paired RDDs

are nothing but RDDs containing a key-value pair. Basically, key-value pair (KVP) consists of a two linked data item in it. Here, the key is the identifier, whereas value is the data corresponding to the key value.

There are several ways to create Paired RDD in Spark, like by running a map() function that returns key-value pairs.

In [ ]:
val lines22 = sc.textFile("data/eternalflame.txt")
val pairs22= lines22.map(s => (s, 1))
val counts1 = pairs22.reduceByKey((a, b) => a + b).sortByKey()
counts1.collect()

In [ ]:
// Let's create Spark paired RDD
val pairs = lines22.map(x => (x.split("")(0), x))
pairs.take(3)

#### Transformation Operations

In [ ]:
pairs.groupByKey().collect()

In [ ]:
pairs.reduceByKey((x, y) => x+y).collect

In [ ]:
// combineByKey

In [ ]:
pairs.mapValues(x => x+1).collect()

In [ ]:
pairs.keys.collect()

In [ ]:
pairs.values.take(5)

In [ ]:
pairs.sortByKey().collect()

##### Actions Operations

In [ ]:
pairs.countByKey()

In [ ]:
pairs.collectAsMap()

In [ ]:
pairs.lookup("D")

In [ ]:
// map(func)

// The map function iterates over every line in RDD and split into new RDD. Using map() transformation we take in any function, and that function is applied to every element of RDD.

In [ ]:
val data = spark.read.textFile("data/eternalflame.txt").rdd

In [ ]:
val mapFile = data.map(line => (line, line.length))
mapFile.foreach(println)

In [ ]:
// flatMap()
// The key difference between map() and flatMap() is map() returns only one element, while flatMap() can return a list of elements.

val flatmapFile = data.flatMap(lines => lines.split(" "))
flatmapFile.foreach(println)


In [ ]:
// filter(func)

// Spark RDD filter() function returns a new RDD, containing only the elements that meet a predicate.

val mapFile = data.flatMap(lines=> lines.split(" ")).filter(value => value=="heart")
mapFile.collect()

In [ ]:
// mapPartitions(func)
// The MapPartition converts each partition of the source RDD into many elements of the result (possibly none). In mapPartition(), the map() function is applied on each partitions simultaneously. MapPartition is like a map, but the difference is it runs separately on each partition(block) of the RDD

In [ ]:
// mapPartitionWithIndex()

In [ ]:
// Union

val rdd1 = spark.sparkContext.parallelize(Seq((1,"jan",2016),(3,"nov",2014),(16,"feb",2014)))
val rdd2 = spark.sparkContext.parallelize(Seq((5,"dec",2014),(17,"sep",2015)))
val rdd3 = spark.sparkContext.parallelize(Seq((6,"dec",2011),(16,"may",2015)))
val rddUnion = rdd1.union(rdd2).union(rdd3)
rddUnion.foreach(println)

In [ ]:
// Intersection

val rdd1 = spark.sparkContext.parallelize(Seq((1,"jan",2016),(3,"nov",2014), (16,"feb",2014)  ))
val rdd2 = spark.sparkContext.parallelize(Seq((5,"dec",2014),(1,"jan",2016)))
val comman = rdd1.intersection(rdd2)
comman.foreach(println)

In [ ]:
// distinct()

val rdd1 = spark.sparkContext.parallelize(Seq((1,"jan",2016),(3,"nov",2014),(16,"feb",2014),(3,"nov",2014)))
val result = rdd1.distinct()
println(result.collect().mkString(", "))


In [ ]:
// groupByKey()

val data = spark.sparkContext.parallelize(Array(('k',5),('s',3),('s',4),('p',7),('p',5),('t',8),('k',6)),3)
val group = data.groupByKey().collect()
group.foreach(println)

In [ ]:
// reduceByKey()
val words = Array("one","two","two","four","five","six","six","eight","nine","ten")
val data = spark.sparkContext.parallelize(words).map(w => (w,1)).reduceByKey(_+_)
data.foreach(println)

In [ ]:
// sortByKey
val data = spark.sparkContext.parallelize(Seq(("maths",52), ("english",75), ("science",82), ("computer",65), ("maths",85)))
val sorted = data.sortByKey()
sorted.foreach(println)

In [ ]:
// join()


// The Join is database term. It combines the fields from two table using common values. join() operation in Spark is defined on pair-wise RDD. Pair-wise RDDs are RDD in which each element is in the form of tuples. Where the first element is key and the second element is the value.

val data = spark.sparkContext.parallelize(Array(('A',1),('b',2),('c',3)))
val data2 =spark.sparkContext.parallelize(Array(('A',4),('A',6),('b',7),('c',3),('c',8)))
val result = data.join(data2)
println(result.collect().mkString(","))

In [ ]:
// coalesce() 
// To avoid full shuffling of data we use coalesce() function. In coalesce() we use existing partition so that less data is shuffled.

val rdd1 = spark.sparkContext.parallelize(Array("jan","feb","mar","april","may","jun"),3)
val result = rdd1.coalesce(2)
result.foreach(println)

In [ ]:
// .count()
val data = spark.read.textFile("data/eternalflame.txt").rdd
val mapFile = data.flatMap(lines => lines.split(" ")).filter(value => value=="heart")
println(mapFile.count())

In [ ]:
// collect()
// The action collect() is the common and simplest operation that returns our entire RDDs content to driver program
val data = spark.sparkContext.parallelize(Array(('A',1),('b',2),('c',3)))
val data2 =spark.sparkContext.parallelize(Array(('A',4),('A',6),('b',7),('c',3),('c',8)))
val result = data.join(data2)
println(result.collect().mkString(","))

In [ ]:
// take()
val data = spark.sparkContext.parallelize(Array(('k',5),('s',3),('s',4),('p',7),('p',5),('t',8),('k',6)),3)
val group = data.groupByKey().collect()
val twoRec = result.take(2)
twoRec.foreach(println)

In [ ]:
// top()
// not working
val data = spark.read.textFile("data/eternalFlame.txt").rdd
val mapFile = data.map(line => (line,line.length))
val res = mapFile.top(3)
res.foreach(println)

In [ ]:
data.top(3)

In [ ]:
//countByValue

val result= data.map(line => (line,line.length)).countByValue()
result.foreach(println)

In [ ]:
// reduce
// The reduce() function takes the two elements as input from the RDD and then produces the output of the same type as that of the input elements. The simple forms of such function are an addition. We can add the elements of RDD, count the number of words. It accepts commutative and associative operations as an argument.


val rdd1 = spark.sparkContext.parallelize(List(20,32,45,62,8,5))
val sum = rdd1.reduce(_+_)
println(sum)

In [ ]:
// fold()
// The signature of the fold() is like reduce(). Besides, it takes “zero value” as input, which is used for the initial call on each partition. But, the condition with zero value is that it should be the identity element of that operation. The key difference between fold() and reduce() is that, reduce() throws an exception for empty collection, but fold() is defined for empty collection.

val rdd1 = spark.sparkContext.parallelize(List(("maths", 80),("science", 90)))
val additionalMarks = ("extra", 4)
val sum = rdd1.fold(additionalMarks){ (acc, marks) => val add = acc._2 + marks._2
("total", add)
}
println(sum)

In [ ]:
// aggregate

In [ ]:
// foreach
// When we have a situation where we want to apply operation on each element of RDD, but it should not return value to the driver.

val data = spark.sparkContext.parallelize(Array(('k',5),('s',3),('s',4),('p',7),('p',5),('t',8),('k',6)),3)
val group = data.groupByKey().collect()
group.foreach(println)

In [ ]:
rdd.getClass

In [ ]:
rdd.take(1)

In [ ]:
val data = Array(1, 2, 3, 4, 5)
val distData = sc.parallelize(data)

In [ ]:
distData.reduce((a, b) => a + b)

In [ ]:
// distData.map(s => s.length).reduce((a, b) => a + b)

In [ ]:
// val lines = sc.textFile("data.txt")

In [ ]:
// val lineLengths = lines.map(s => s.length)
// linesLengths

In [ ]:
// val totalLength = lineLengths.reduce((a, b) => a + b)
// print(totalLength)

In [ ]:
// Working with Key-Value Pairs

In [ ]:
// val pairs = lines.map(s => (s, 1))
// val counts = pairs.reduceByKey((a, b) => a + b)

In [ ]:
// pairs.take(2)

In [ ]:
// val counts = pairs.reduceByKey((a, b) => a + b)

In [ ]:
// print(counts.count())

In [ ]:
// lines.sample(withReplacement= true , fraction=0.2, 1 ).take(2)

# Samples 
from https://databricks-prod-cloudfront.cloud.databricks.com/public/4027ec902e239c93eaaa8714f173bcfc/9000630084352253/509646307872265/6405721786982255/latest.html

### Map Operations

In [5]:
//1. map()-  Return a new RDD by applying a function to each element of this RDD
//map(func) 

val x = sc.parallelize(Array("b", "a", "c"))
val y = x.map(z => (z,1))
println(x.collect().mkString(", "))
println(y.collect().mkString(", "))

collect at cmd4.sc:3

2 / 2

b, a, c


collect at cmd4.sc:4

2 / 2

(b,1), (a,1), (c,1)


x: org.apache.spark.rdd.RDD[String] = ParallelCollectionRDD[0] at parallelize at cmd4.sc:1
y: org.apache.spark.rdd.RDD[(String, Int)] = MapPartitionsRDD[1] at map at cmd4.sc:2

In [6]:
//2. flatmap()- Return a new RDD by first applying a function to all elements of this RDD, and then flattening the results
//flatMap(func)  

val x = sc.parallelize(Array(1,2,3))
val y = x.flatMap(n => Array(n, n*100, 42))
println(x.collect().mkString(", "))
println(y.collect().mkString(", "))

collect at cmd5.sc:3

2 / 2

1, 2, 3


collect at cmd5.sc:4

2 / 2

1, 100, 42, 2, 200, 42, 3, 300, 42


x: org.apache.spark.rdd.RDD[Int] = ParallelCollectionRDD[2] at parallelize at cmd5.sc:1
y: org.apache.spark.rdd.RDD[Int] = MapPartitionsRDD[3] at flatMap at cmd5.sc:2

In [13]:
x.flatMap(n => Array(n, n - 1)).collect().foreach(println)

collect at cmd12.sc:1

2 / 2

1
0
2
1
3
2


In [16]:
x.map(z => z -1).collect()

collect at cmd15.sc:1

2 / 2

res15: Array[Int] = Array(0, 1, 2)

In [17]:
//3. mapPartitions()- Return a new RDD by applying a function to each partition of this RDD
//similar to map 
//mapPartitions(func) 

val parallel = sc.parallelize(1 to 9, 3)
parallel.mapPartitions( x => List(x.next).iterator).collect

collect at cmd16.sc:2

3 / 3

parallel: org.apache.spark.rdd.RDD[Int] = ParallelCollectionRDD[14] at parallelize at cmd16.sc:1
res16_1: Array[Int] = Array(1, 4, 7)

### Set Operations

In [21]:
//union()- Return a new RDD containing all items from two original RDDs. Duplicates are not culled
//union(otherDataset) 
//glom() flattens elements on the same partition

val x= sc.parallelize(Array(1,2,3), 2)
val y= sc.parallelize(Array(3,4), 1)
val z= x.union(y)
print(z.collect().mkString(", "))
val zOut= z.glom().collect()
val zout1= z.collect()

collect at cmd20.sc:4

3 / 3

1, 2, 3, 3, 4

collect at cmd20.sc:5

3 / 3

collect at cmd20.sc:6

3 / 3

x: org.apache.spark.rdd.RDD[Int] = ParallelCollectionRDD[20] at parallelize at cmd20.sc:1
y: org.apache.spark.rdd.RDD[Int] = ParallelCollectionRDD[21] at parallelize at cmd20.sc:2
z: org.apache.spark.rdd.RDD[Int] = UnionRDD[22] at union at cmd20.sc:3
zOut: Array[Array[Int]] = Array(Array(1), Array(2, 3), Array(3, 4))
zout1: Array[Int] = Array(1, 2, 3, 3, 4)

In [22]:
//distinct()- Return a new RDD containing distinct items from the original RDD (omitting all duplicates)
//distinct([numPartitions]))

val x= sc.parallelize(Array(1,2,3,3,4))
val y= x.distinct()
println(y.collect().mkString(", "))

distinct at cmd21.sc:2

2 / 2

collect at cmd21.sc:3

2 / 2

4, 2, 1, 3


x: org.apache.spark.rdd.RDD[Int] = ParallelCollectionRDD[24] at parallelize at cmd21.sc:1
y: org.apache.spark.rdd.RDD[Int] = MapPartitionsRDD[27] at distinct at cmd21.sc:2

In [23]:
//intersection() :  Returns elements that are common b/w both RDDs. Also removed Duplicates
//intersection(otherDataset) 

val x = sc.parallelize(List("lion", "tiger", "tiger", "peacock", "horse"))
val y = sc.parallelize(List("lion", "tiger"))
x.intersection(y).collect()


intersection at cmd22.sc:3

2 / 2

intersection at cmd22.sc:3

2 / 2

collect at cmd22.sc:3

2 / 2

x: org.apache.spark.rdd.RDD[String] = ParallelCollectionRDD[28] at parallelize at cmd22.sc:1
y: org.apache.spark.rdd.RDD[String] = ParallelCollectionRDD[29] at parallelize at cmd22.sc:2
res22_2: Array[String] = Array("lion", "tiger")

In [24]:
//subtract()- Returns only elements that are present in the first RDD

val x = sc.parallelize(List("lion", "tiger", "tiger", "peacock", "horse"))
val y = sc.parallelize(List("lion", "tiger"))
x.subtract(y).collect()

subtract at cmd23.sc:3

2 / 2

subtract at cmd23.sc:3

2 / 2

collect at cmd23.sc:3

2 / 2

x: org.apache.spark.rdd.RDD[String] = ParallelCollectionRDD[36] at parallelize at cmd23.sc:1
y: org.apache.spark.rdd.RDD[String] = ParallelCollectionRDD[37] at parallelize at cmd23.sc:2
res23_2: Array[String] = Array("peacock", "horse")

In [25]:
//cartesian()- Provides cartesian product b/w 2 RDDs
//cartesian(otherDataset) 

val x = sc.parallelize(List("lion", "tiger", "tiger", "peacock", "horse"))
val y = sc.parallelize(List("lion", "tiger"))
x.cartesian(y).collect()

collect at cmd24.sc:3

4 / 4

x: org.apache.spark.rdd.RDD[String] = ParallelCollectionRDD[42] at parallelize at cmd24.sc:1
y: org.apache.spark.rdd.RDD[String] = ParallelCollectionRDD[43] at parallelize at cmd24.sc:2
res24_2: Array[(String, String)] = Array(
  ("lion", "lion"),
  ("tiger", "lion"),
  ("lion", "tiger"),
  ("tiger", "tiger"),
  ("tiger", "lion"),
  ("peacock", "lion"),
  ("horse", "lion"),
  ("tiger", "tiger"),
  ("peacock", "tiger"),
  ("horse", "tiger")
)

### Other Operations

In [26]:
//filter
//filter(func)- Return a new dataset formed by selecting those elements of the source on which func returns true. 
val x = sc.parallelize(Array(1,2,3))
val y = x.filter(n => n%2 == 1)
println(x.collect().mkString(", "))
println(y.collect().mkString(", "))

collect at cmd25.sc:3

2 / 2

1, 2, 3


collect at cmd25.sc:4

2 / 2

1, 3


x: org.apache.spark.rdd.RDD[Int] = ParallelCollectionRDD[45] at parallelize at cmd25.sc:1
y: org.apache.spark.rdd.RDD[Int] = MapPartitionsRDD[46] at filter at cmd25.sc:2

In [27]:
//groupby
val x= sc.parallelize(Array("John", "Fred", "Anna", "James"))
val y= x.groupBy(w => w.charAt(0))
println(x.collect().mkString(", "))
println(y.collect().mkString(", "))

collect at cmd26.sc:3

2 / 2

John, Fred, Anna, James


groupBy at cmd26.sc:2

2 / 2

collect at cmd26.sc:4

2 / 2

(J,CompactBuffer(John, James)), (F,CompactBuffer(Fred)), (A,CompactBuffer(Anna))


x: org.apache.spark.rdd.RDD[String] = ParallelCollectionRDD[47] at parallelize at cmd26.sc:1
y: org.apache.spark.rdd.RDD[(Char, Iterable[String])] = ShuffledRDD[49] at groupBy at cmd26.sc:2

In [28]:
y.collect()

collect at cmd27.sc:1

2 / 2

res27: Array[(Char, Iterable[String])] = Array(
  ('J', CompactBuffer("John", "James")),
  ('F', CompactBuffer("Fred")),
  ('A', CompactBuffer("Anna"))
)

In [29]:
//toDebugString 

val x = sc.parallelize(List("This is a word", "This is another word"), 7)
val y = x.flatMap(line => line.split(" "))
val z = y.map ( word => (word, 1) )
val a = z.reduceByKey ( (x, y) => x + y )
a.toDebugString
a.collect()


map at cmd28.sc:3

7 / 7

collect at cmd28.sc:6

7 / 7

x: org.apache.spark.rdd.RDD[String] = ParallelCollectionRDD[50] at parallelize at cmd28.sc:1
y: org.apache.spark.rdd.RDD[String] = MapPartitionsRDD[51] at flatMap at cmd28.sc:2
z: org.apache.spark.rdd.RDD[(String, Int)] = MapPartitionsRDD[52] at map at cmd28.sc:3
a: org.apache.spark.rdd.RDD[(String, Int)] = ShuffledRDD[53] at reduceByKey at cmd28.sc:4
res28_4: String = """(7) ShuffledRDD[53] at reduceByKey at cmd28.sc:4 []
 +-(7) MapPartitionsRDD[52] at map at cmd28.sc:3 []
    |  MapPartitionsRDD[51] at flatMap at cmd28.sc:2 []
    |  ParallelCollectionRDD[50] at parallelize at cmd28.sc:1 []"""
res28_5: Array[(String, Int)] = Array(
  ("This", 2),
  ("is", 2),
  ("another", 1),
  ("a", 1),
  ("word", 2)
)

In [31]:
y.collect()

collect at cmd30.sc:1

7 / 7

res30: Array[String] = Array(
  "This",
  "is",
  "a",
  "word",
  "This",
  "is",
  "another",
  "word"
)

### PairRDD Functions (Single RDD)

In [32]:
//groupbykey - When called on a dataset of (K, V) pairs, returns a dataset of (K, Iterable<V>) pairs
//groupByKey([numPartitions]) 

val x= sc.parallelize(Array(('B',5),('B',4),('A',3),('A',2),('A',1)))
val y= x.groupByKey()
println(x.collect().mkString(", "))
println(y.collect().mkString(", "))

collect at cmd31.sc:3

2 / 2

(B,5), (B,4), (A,3), (A,2), (A,1)


parallelize at cmd31.sc:1

2 / 2

collect at cmd31.sc:4

2 / 2

(B,CompactBuffer(5, 4)), (A,CompactBuffer(3, 2, 1))


x: org.apache.spark.rdd.RDD[(Char, Int)] = ParallelCollectionRDD[54] at parallelize at cmd31.sc:1
y: org.apache.spark.rdd.RDD[(Char, Iterable[Int])] = ShuffledRDD[55] at groupByKey at cmd31.sc:2

In [33]:
//keyBy- Create a Pair RDD, forming one pair for each item in the original RDD. The pair’s key is calculated from the value via a user-supplied function.

val x= sc.parallelize(Array("John", "Fred", "Anna", "James"))
val y= x.keyBy(w => w.charAt(0))
println(x.collect().mkString(", "))
println(y.collect().mkString(", "))

collect at cmd32.sc:3

2 / 2

John, Fred, Anna, James


collect at cmd32.sc:4

2 / 2

(J,John), (F,Fred), (A,Anna), (J,James)


x: org.apache.spark.rdd.RDD[String] = ParallelCollectionRDD[56] at parallelize at cmd32.sc:1
y: org.apache.spark.rdd.RDD[(Char, String)] = MapPartitionsRDD[57] at keyBy at cmd32.sc:2

### PairRDD Functions (Two RDD)

In [34]:
//join - Return a new RDD containing all pairs of elements having the same key in the original RDDs
//join(otherDataset, [numPartitions])

val x= sc.parallelize(Seq(("math", 55),("math", 56),("english", 57),("english", 58),("science", 59),("science", 54)))
val y= sc.parallelize(Seq(("math", 60),("math", 65),("science", 61),("science", 62),("history", 63),("history", 64)))
val z= x.join(y)
println(z.collect().mkString(", "))

parallelize at cmd33.sc:2

2 / 2

parallelize at cmd33.sc:1

2 / 2

collect at cmd33.sc:4

2 / 2

(math,(55,60)), (math,(55,65)), (math,(56,60)), (math,(56,65)), (science,(59,61)), (science,(59,62)), (science,(54,61)), (science,(54,62))


x: org.apache.spark.rdd.RDD[(String, Int)] = ParallelCollectionRDD[58] at parallelize at cmd33.sc:1
y: org.apache.spark.rdd.RDD[(String, Int)] = ParallelCollectionRDD[59] at parallelize at cmd33.sc:2
z: org.apache.spark.rdd.RDD[(String, (Int, Int))] = MapPartitionsRDD[62] at join at cmd33.sc:3

In [36]:
//leftOuterJoin() 
val x= sc.parallelize(Seq(("math", 55),("math", 56),("english", 57),("english", 58),("science", 59),("science", 54)))
val y= sc.parallelize(Seq(("math", 60),("math", 65),("science", 61),("science", 62),("history", 63),("history", 64)))
val z= x.leftOuterJoin(y)
println(z.collect().mkString(", "))


parallelize at cmd35.sc:1

2 / 2

parallelize at cmd35.sc:2

2 / 2

collect at cmd35.sc:4

2 / 2

(math,(55,Some(60))), (math,(55,Some(65))), (math,(56,Some(60))), (math,(56,Some(65))), (science,(59,Some(61))), (science,(59,Some(62))), (science,(54,Some(61))), (science,(54,Some(62))), (english,(57,None)), (english,(58,None))


x: org.apache.spark.rdd.RDD[(String, Int)] = ParallelCollectionRDD[68] at parallelize at cmd35.sc:1
y: org.apache.spark.rdd.RDD[(String, Int)] = ParallelCollectionRDD[69] at parallelize at cmd35.sc:2
z: org.apache.spark.rdd.RDD[(String, (Int, Option[Int]))] = MapPartitionsRDD[72] at leftOuterJoin at cmd35.sc:3

In [37]:
//rightOuterJoin() 

val x= sc.parallelize(Seq(("math", 55),("math", 56),("english", 57),("english", 58),("science", 59),("science", 54)))
val y= sc.parallelize(Seq(("math", 60),("math", 65),("science", 61),("science", 62),("history", 63),("history", 64)))
val z= x.rightOuterJoin(y)
println(z.collect().mkString(", "))

parallelize at cmd36.sc:1

2 / 2

parallelize at cmd36.sc:2

2 / 2

collect at cmd36.sc:4

2 / 2

(math,(Some(55),60)), (math,(Some(55),65)), (math,(Some(56),60)), (math,(Some(56),65)), (history,(None,63)), (history,(None,64)), (science,(Some(59),61)), (science,(Some(59),62)), (science,(Some(54),61)), (science,(Some(54),62))


x: org.apache.spark.rdd.RDD[(String, Int)] = ParallelCollectionRDD[73] at parallelize at cmd36.sc:1
y: org.apache.spark.rdd.RDD[(String, Int)] = ParallelCollectionRDD[74] at parallelize at cmd36.sc:2
z: org.apache.spark.rdd.RDD[(String, (Option[Int], Int))] = MapPartitionsRDD[77] at rightOuterJoin at cmd36.sc:3

In [38]:
//cogroup - Multiple Pair RDDs can be combined using cogroup()
//cogroup(otherDataset, [numPartitions]) 

val x= sc.parallelize(Array(("a", 1), ("b", 2)))
val y= sc.parallelize(Array(("a", 3), ("a", 4), ("b", 5)))
val z= x.cogroup(y)
println(z.collect().mkString(", "))

parallelize at cmd37.sc:1

2 / 2

parallelize at cmd37.sc:2

2 / 2

collect at cmd37.sc:4

2 / 2

(b,(CompactBuffer(2),CompactBuffer(5))), (a,(CompactBuffer(1),CompactBuffer(3, 4)))


x: org.apache.spark.rdd.RDD[(String, Int)] = ParallelCollectionRDD[78] at parallelize at cmd37.sc:1
y: org.apache.spark.rdd.RDD[(String, Int)] = ParallelCollectionRDD[79] at parallelize at cmd37.sc:2
z: org.apache.spark.rdd.RDD[(String, (Iterable[Int], Iterable[Int]))] = MapPartitionsRDD[81] at cogroup at cmd37.sc:3

### Sorting

In [39]:
//sortByKey()- part of OrderedRDDFunctions that works on Key/Value pairs.
//sortByKey([ascending], [numPartitions])

val x= sc.parallelize(Seq(("math", 55),("math", 56),("english", 57),("english", 58),("science", 59),("science", 54)))
x.sortByKey().collect()

sortByKey at cmd38.sc:2

2 / 2

parallelize at cmd38.sc:1

2 / 2

collect at cmd38.sc:2

2 / 2

x: org.apache.spark.rdd.RDD[(String, Int)] = ParallelCollectionRDD[82] at parallelize at cmd38.sc:1
res38_1: Array[(String, Int)] = Array(
  ("english", 57),
  ("english", 58),
  ("math", 55),
  ("math", 56),
  ("science", 59),
  ("science", 54)
)

In [40]:
//top() & takeOrdered() are actions that return the N elements based on the default ordering or the Customer ordering provided by us
//takeOrdered(n, [ordering]) 

val x = sc.parallelize{ Seq(10, 4, 5, 3, 11, 2, 6) }
val xout = x.top(5)
val yout = x.takeOrdered(5)
println(xout)
println(yout)

top at cmd39.sc:2

2 / 2

takeOrdered at cmd39.sc:3

2 / 2

[I@4a0ade30
[I@37e6e78c


x: org.apache.spark.rdd.RDD[Int] = ParallelCollectionRDD[86] at parallelize at cmd39.sc:1
xout: Array[Int] = Array(11, 10, 6, 5, 4)
yout: Array[Int] = Array(2, 3, 4, 5, 6)

### Partition

In [41]:
//COALESCE - Return a new RDD which is reduced to a smaller number of partitions
//coalesce(numPartitions)

val x= sc.parallelize(Array(1, 2, 3, 4, 5), 3)
val y= x.coalesce(2)
val xOut= x.glom().collect()
val yOut= y.glom().collect()
val xOut1= x.collect()
val yOut2= y.collect()

collect at cmd40.sc:3

3 / 3

collect at cmd40.sc:4

2 / 2

collect at cmd40.sc:5

3 / 3

collect at cmd40.sc:6

2 / 2

x: org.apache.spark.rdd.RDD[Int] = ParallelCollectionRDD[89] at parallelize at cmd40.sc:1
y: org.apache.spark.rdd.RDD[Int] = CoalescedRDD[90] at coalesce at cmd40.sc:2
xOut: Array[Array[Int]] = Array(Array(1), Array(2, 3), Array(4, 5))
yOut: Array[Array[Int]] = Array(Array(1), Array(2, 3, 4, 5))
xOut1: Array[Int] = Array(1, 2, 3, 4, 5)
yOut2: Array[Int] = Array(1, 2, 3, 4, 5)

In [42]:
//partition by- Return a new RDD with the specified number of partitions, placing original items into the partition returned by a user supplied function

import org.apache.spark.Partitioner
val x= sc.parallelize(Array(('J',"James"),('F',"Fred"),('A',"Anna"),('J',"John")), 3)
val y= x.partitionBy(new Partitioner() { 
  val numPartitions= 2
  def getPartition(k:Any) = {
  if (k.asInstanceOf[Char] < 'H') 0 else 1 
  }
})
val xOut= x.glom().collect()
val yOut= y.glom().collect()


collect at cmd41.sc:9

3 / 3

parallelize at cmd41.sc:2

3 / 3

collect at cmd41.sc:10

2 / 2

import org.apache.spark.Partitioner

x: org.apache.spark.rdd.RDD[(Char, String)] = ParallelCollectionRDD[93] at parallelize at cmd41.sc:2
y: org.apache.spark.rdd.RDD[(Char, String)] = ShuffledRDD[94] at partitionBy at cmd41.sc:3
xOut: Array[Array[(Char, String)]] = Array(
  Array(('J', "James")),
  Array(('F', "Fred")),
  Array(('A', "Anna"), ('J', "John"))
)
yOut: Array[Array[(Char, String)]] = Array(
  Array(('F', "Fred"), ('A', "Anna")),
  Array(('J', "James"), ('J', "John"))
)

In [43]:
//zip - Return a new RDD containing pairs whose key is the item in the original RDD, and whose value is that item’s corresponding element (same partition, same index) in a second RDD
// zip(otherRDD)

val x= sc.parallelize(Array(1,2,3))
val y= x.map(n=>n*n)
val z= x.zip(y)
println(z.collect().mkString(", "))



collect at cmd42.sc:4

2 / 2

(1,1), (2,4), (3,9)


x: org.apache.spark.rdd.RDD[Int] = ParallelCollectionRDD[97] at parallelize at cmd42.sc:1
y: org.apache.spark.rdd.RDD[Int] = MapPartitionsRDD[98] at map at cmd42.sc:2
z: org.apache.spark.rdd.RDD[(Int, Int)] = ZippedPartitionsRDD2[99] at zip at cmd42.sc:3

#### Actions

In [44]:
// getnumpartitions - Return the number of partitions in RDD

val x= sc.parallelize(Array(1,2,3), 2)
val y= x.partitions.size
val xOut= x.glom().collect()
println(y)


collect at cmd43.sc:3

2 / 2

2


x: org.apache.spark.rdd.RDD[Int] = ParallelCollectionRDD[100] at parallelize at cmd43.sc:1
y: Int = 2
xOut: Array[Array[Int]] = Array(Array(1), Array(2, 3))

In [45]:
//collect - Return all items in the RDD to the driver in a single list

val x= sc.parallelize(Array(1,2,3), 2)
val y= x.collect()
val xOut= x.glom().collect()
println(y)

collect at cmd44.sc:2

2 / 2

collect at cmd44.sc:3

2 / 2

[I@7ee57bbb


x: org.apache.spark.rdd.RDD[Int] = ParallelCollectionRDD[102] at parallelize at cmd44.sc:1
y: Array[Int] = Array(1, 2, 3)
xOut: Array[Array[Int]] = Array(Array(1), Array(2, 3))

In [46]:
//reduce - Aggregate all the elements of the RDD by applying a user function pairwise to elements and partial results, and returns a result to the driver
//reduce(func) 

val x= sc.parallelize(Array(1,2,3,4))
val y= x.reduce((a,b) => a+b)
println(x.collect.mkString(", "))
println(y)

reduce at cmd45.sc:2

2 / 2

collect at cmd45.sc:3

2 / 2

1, 2, 3, 4
10


x: org.apache.spark.rdd.RDD[Int] = ParallelCollectionRDD[104] at parallelize at cmd45.sc:1
y: Int = 10

In [47]:
//aggregate - Aggregate all the elements of the RDD by: 
// -applying a user function to combine elements with user-supplied objects, 
//-then combining those user-defined results via a second user function, 
//-and finally returning a result to the driver.

def seqOp= (data:(Array[Int], Int), item:Int) => (data._1 :+ item, data._2 + item)
def combOp= (d1:(Array[Int], Int), d2:(Array[Int], Int)) => (d1._1.union(d2._1), d1._2 + d2._2)
val x= sc.parallelize(Array(1,2,3,4))
val y= x.aggregate((Array[Int](), 0))(seqOp, combOp)
println(y)

aggregate at cmd46.sc:4

2 / 2

([I@4a3f4139,10)


defined function seqOp
defined function combOp
x: org.apache.spark.rdd.RDD[Int] = ParallelCollectionRDD[105] at parallelize at cmd46.sc:3
y: (Array[Int], Int) = (Array(3, 4, 1, 2), 10)

In [48]:
//max - Return the maximum item in the RDD

val x= sc.parallelize(Array(2,4,1))
val y= x.max
println(x.collect().mkString(", "))
println(y)

max at cmd47.sc:2

2 / 2

collect at cmd47.sc:3

2 / 2

2, 4, 1
4


x: org.apache.spark.rdd.RDD[Int] = ParallelCollectionRDD[106] at parallelize at cmd47.sc:1
y: Int = 4

In [49]:
//sum - Return the sum of the items in the RDD

val x= sc.parallelize(Array(2,4,1))
val y= x.sum
println(x.collect().mkString(", "))
println(y)

sum at cmd48.sc:2

2 / 2

collect at cmd48.sc:3

2 / 2

2, 4, 1
7.0


x: org.apache.spark.rdd.RDD[Int] = ParallelCollectionRDD[107] at parallelize at cmd48.sc:1
y: Double = 7.0

In [50]:
//mean - Return the mean of the items in the RDD

val x= sc.parallelize(Array(2,4,1))
val y= x.mean
println(x.collect().mkString(", "))
println(y)

mean at cmd49.sc:2

2 / 2

collect at cmd49.sc:3

2 / 2

2, 4, 1
2.3333333333333335


x: org.apache.spark.rdd.RDD[Int] = ParallelCollectionRDD[109] at parallelize at cmd49.sc:1
y: Double = 2.3333333333333335

In [51]:
//stddev- Return the standard deviation of the items in the RDD

val x= sc.parallelize(Array(2,4,1))
val y= x.stdev
println(x.collect().mkString(", "))
println(y)

stdev at cmd50.sc:2

2 / 2

collect at cmd50.sc:3

2 / 2

2, 4, 1
1.247219128924647


x: org.apache.spark.rdd.RDD[Int] = ParallelCollectionRDD[112] at parallelize at cmd50.sc:1
y: Double = 1.247219128924647

In [52]:
//countByKey- Return a map of keys and counts of their occurrences in the RDD

val x= sc.parallelize(Array(('J',"James"),('F',"Fred"),('A',"Anna"),('J',"John")))
val y= x.countByKey()
println(y)

countByKey at cmd51.sc:2

2 / 2

countByKey at cmd51.sc:2

2 / 2

Map(J -> 2, F -> 1, A -> 1)


x: org.apache.spark.rdd.RDD[(Char, String)] = ParallelCollectionRDD[115] at parallelize at cmd51.sc:1
y: collection.Map[Char, Long] = Map('J' -> 2L, 'F' -> 1L, 'A' -> 1L)

In [ ]:
// //saveAsTextFile - Save the RDD to the filesystemindicated in the path
// //saveAsTextFile(path) 

// dbutils.fs.rm("/temp/demo", true)
// val x= sc.parallelize(Array(2,4,1))
// x.saveAsTextFile("/temp/demo")
// val y= sc.textFile("/temp/demo")
// println(y.collect().mkString(", "))